In [2]:
import pandas as pd
import numpy as np

# 1. 데이터 불러오기
try:
    df = pd.read_csv('../LEI/data/seoul_culture_event.csv')

    # 2. 데이터 전처리
    # 필요한 컬럼(자치구, 분류)에 결측치가 있는 행은 분석에서 제외합니다.
    df_cleaned = df.dropna(subset=['자치구', '분류'])
    # '자치구' 컬럼에서 '서울' 또는 공백 값을 가진 불필요한 데이터를 제거합니다.
    df_cleaned = df_cleaned[~df_cleaned['자치구'].isin(['서울', ' '])]


    # 3. 샤논 다양성 지수 계산 함수 정의
    def calculate_shannon_diversity(series):
        """
        데이터 시리즈를 받아 각 항목의 비율을 기반으로
        샤논 다양성 지수를 계산합니다.
        """
        counts = series.value_counts()
        proportions = counts / counts.sum()
        shannon_index = -np.sum(proportions * np.log(proportions))
        return shannon_index


    # 4. 구별 데이터 분석 및 여가도 계산
    # '자치구'를 기준으로 그룹화하여 분석을 수행합니다.
    leisure_analysis = df_cleaned.groupby('자치구').agg(
        # '분류' 컬럼의 개수를 세어 '총 행사 수'를 계산합니다.
        총_행사_수=('분류', 'count'),
        # 위에서 정의한 함수를 이용해 '행사 다양성 지수'를 계산합니다.
        행사_다양성_지수=('분류', calculate_shannon_diversity)
    ).reset_index()

    # 5. 최종 '여가도 점수' 산출
    # 제안된 공식을 적용하여 최종 점수를 계산합니다.
    leisure_analysis['여가도_점수'] = leisure_analysis['총_행사_수'] * (1 + leisure_analysis['행사_다양성_지수'])

    # 6. 결과 정렬 및 출력
    # '여가도_점수'가 높은 순으로 정렬하여 결과를 확인하기 쉽게 합니다.
    results_sorted = leisure_analysis.sort_values(by='여가도_점수', ascending=False)
    # 보기 좋게 인덱스를 재설정하고, 소수점 둘째 자리까지만 표시합니다.
    results_sorted = results_sorted.reset_index(drop=True)
    pd.options.display.float_format = '{:.2f}'.format

    print("--- 서울시 자치구별 문화행사 여가도 분석 결과 ---")
    print(results_sorted)
    
    # 8. CSV 파일로 저장 (추가된 부분)
    # 한글이 깨지지 않도록 'utf-8-sig' 인코딩을 사용합니다.
    # index=False는 불필요한 순번(0,1,2...)이 파일에 저장되지 않도록 합니다.
    output_filename = '../LEI/results/seoul_leisure_score_result.csv'
    results_sorted.to_csv(output_filename, encoding='utf-8-sig', index=False)


except FileNotFoundError:
    print("오류: 'seoul_culture_event.csv' 파일을 찾을 수 없습니다.")
except Exception as e:
    print(f"파일을 처리하는 중 오류가 발생했습니다: {e}")

--- 서울시 자치구별 문화행사 여가도 분석 결과 ---
     자치구  총_행사_수  행사_다양성_지수  여가도_점수
0    종로구     434       2.33 1444.45
1     중구     208       1.95  613.01
2    마포구     169       2.12  527.93
3    구로구      96       2.07  294.40
4    은평구     112       1.58  288.59
5    송파구     110       1.55  281.02
6    서초구      93       1.94  273.37
7    도봉구      92       1.67  245.58
8    성동구      76       2.21  243.89
9    노원구      74       2.20  236.57
10   광진구      72       2.08  221.56
11   관악구      63       2.01  189.47
12   강동구      56       2.08  172.21
13  동대문구      68       1.32  158.09
14   강북구      42       2.00  126.21
15   금천구      32       2.18  101.91
16   양천구      31       2.16   98.03
17   강남구      47       0.92   90.38
18   용산구      31       1.34   72.40
19   중랑구      30       1.38   71.38
20  영등포구      17       1.73   46.44
21   동작구      18       1.53   45.57
22  서대문구      15       1.96   44.35
23   성북구      13       1.82   36.64
24   강서구      10       1.64   26.43
